In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from bs4 import BeautifulSoup
import urllib
import shapely
from shapely.ops import cascaded_union
import math
import json
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
# Census data
population_by_dwelling = pd.read_csv('https://www.dropbox.com/s/hd2na0f663jt086/Resident%20Population%20by%20Planning%20Area_Subzone%20and%20Type%20of%20Dwelling.csv?dl=1')
transportation_mode = pd.read_csv('https://www.dropbox.com/s/wgoctuksbcz2zdk/Resident%20Working%20Persons%20Usual%20Mode%20of%20Transport%20to%20Work.csv?dl=1')
population_by_age = pd.read_csv('https://www.dropbox.com/s/bhd8j7a8sxkssfc/POP2015.csv?dl=1')
population_by_race = pd.read_csv('https://www.dropbox.com/s/v53je5tntk3gy73/Resident%20Population%20by%20Planning%20Area_Subzone%2C%20Ethnic%20Group%20and%20Sex.csv?dl=1')
# Spatial data
area = gpd.read_file('Data/SpatialData/singapore_planning_areas.geojson')
area.rename(columns={'Planning_A':'Planning_Area'},inplace=True)

In [3]:
# Prepare for merger
def upper_case(df):
    # Delete observations not containing TOTAL if a TOTAl observation exists
    df['Planning_Area'] = df['Planning_Area'].apply(lambda x: x.replace('-','').upper().replace('TOTAL','').strip())
    df = df.drop_duplicates(subset='Planning_Area')
    return df

In [4]:
population_by_age = upper_case(population_by_age)
transportation_mode = upper_case(transportation_mode)
population_by_dwelling = upper_case(population_by_dwelling)
population_by_race = upper_case(population_by_race)

In [ ]:
#data = area.merge(population_by_age,on='Planning_Area',how='inner')
#data = gpd.GeoDataFrame(data)

In [ ]:
#data.plot()

In [ ]:
# North-East Island above Changi is not inhabitated.
#area.plot()

In [ ]:
#for col in list(data.columns)[2:]:
#    data[col] = np.where(data[col].apply(lambda x: x=='  '),np.nan,data[col])
#    data[col] = data[col].astype(float)
#data['Youth'] = (data[' 0 - 4 '] + data[' 5 - 9 '] + data[' 10 - 14 '] + data[' 15 - 19 '])/data[' Total ']
#data['Elderly'] = (data[' 65 - 69 '] + data[' 70 - 74 '] + data[' 75 - 79 '] + data[' 80 - 84 '] + data[' 85 & Over '])/data[' Total ']

# Modify data in ArcGIS to get only one shapefile for each Planning Area
#with open('data.geojson', 'w') as f:
#    f.write(data.to_json())

In [5]:
# import modified data
# Add race data
#'https://www.dropbox.com/s/jcf1rpvzi4nivn0/planning_area_with_population.geojson?dl=1'
data = gpd.read_file('Data/ProcessedData/planning_area_with_population.geojson')

In [6]:
population_by_race.rename(columns={' Total ':'Total_Population'},inplace=True)
final = data.merge(population_by_race,on='Planning_Area',how='inner')
for i in final.columns:
    final[i] = np.where(final[i].apply(lambda x: x==' na '),np.nan,final[i])
for i in [' Total_Chinese ',  ' Total_Malays ',' Total_Indians  ',  ' Total_Others  ']:
    k = i.strip().replace('Total_','')
    final[k] = final[i].astype(float)/final['Total_Population'].astype(float)    

In [7]:
final = final[['Planning_Area','geometry','Total_Population','Elderly','Youth','Chinese','Malays','Indians','Others']]

In [8]:
# Join data with other file that contains further spatial information obtained by joining the data in ArcGIS
other_data = gpd.read_file('Data/SpatialData/Planning_Area.geojson')

In [9]:
other_data = other_data[['Planning_A','Rec_Bin','W_Facility']]
other_data.rename(columns={'Planning_A':'Planning_Area'},inplace=True)

In [10]:
final = other_data.merge(final,on='Planning_Area',how='inner')

In [13]:
final = gpd.GeoDataFrame(final)

In [14]:
with open('data.geojson', 'w') as f:
    f.write(final.to_json())